<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Oslo_era5_land_hourly_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install "cdsapi>=0.7.7"
!pip install netcdf4
!pip install netcdf4 h5netcdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00


In [2]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

Configuration updated! Now attempting to update the library...


In [3]:
#@title 📍 Select City and View Map { run: "auto" }

# 1. Selection Menu
# Use the dropdown on the right to choose the city
selected_city = "Oslo" #@param ["Turku", "Copenhagen", "Stockholm", "Oslo"]

# 2. City Coordinate Database
city_db = {
    "Turku": {"lat": 60.5, "lon": 22.3, "color": "red"},
    "Copenhagen": {"lat": 55.7, "lon": 12.6, "color": "blue"},
    "Stockholm": {"lat": 59.3, "lon": 18.1, "color": "green"},
    "Oslo": {"lat": 59.9, "lon": 10.7, "color": "purple"}
}

# Get current selection
coords = city_db[selected_city]
print(f"Current Selection: {selected_city} ({coords['lat']}, {coords['lon']})")

# 3. Generate Map showing all options
import folium
m = folium.Map(location=[59.0, 16.0], zoom_start=5, tiles='cartodbpositron')

for city, data in city_db.items():
    is_selected = (city == selected_city)
    folium.Marker(
        [data['lat'], data['lon']],
        popup=f"<b>{city}</b>",
        tooltip=city,
        icon=folium.Icon(color=data['color'], icon='star' if is_selected else 'info-sign')
    ).add_to(m)

m

Current Selection: Oslo (59.9, 10.7)


In [4]:
import cdsapi
import os
import calendar

# Setting up the variables based on your selection above
LAT = city_db[selected_city]['lat']
LON = city_db[selected_city]['lon']
folder_city = selected_city.lower()

c = cdsapi.Client(progress=False)
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

print(f"--- 📥 Downloading data for: {selected_city} ---")

for i, (year, month) in enumerate(tasks):
    filename = f'{folder_city}_{year}_{month}.nc'

    if os.path.exists(filename):
        continue

    # Date logic for 2024 vs 2025
    last_day = 1 if year == '2025' else calendar.monthrange(int(year), int(month))[1]
    days = [str(d).zfill(2) for d in range(1, last_day + 1)]

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': ['2m_temperature', '10m_u_component_of_wind',
                             '10m_v_component_of_wind', 'surface_solar_radiation_downwards'],
                'year': year, 'month': month, 'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [LAT, LON, LAT, LON],
                'format': 'netcdf',
            },
            filename)
        print(f"✅ {filename} done.")
    except Exception as e:
        print(f"❌ Failed {year}-{month}: {e}")

print(f"--- 🎉 All {selected_city} downloads complete ---")

2025-12-20 10:31:50,757 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


--- 📥 Downloading data for: Oslo ---


2025-12-20 10:31:51,104 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 10:31:51,107

✅ oslo_2024_01.nc done.


2025-12-20 10:32:15,157 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 10:32:15,159

...

✅ oslo_2024_11.nc done.


2025-12-20 10:37:06,564 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 10:37:06,566

✅ oslo_2024_12.nc done.


2025-12-20 10:37:30,289 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 10:37:30,291

✅ oslo_2025_01.nc done.
--- 🎉 All Oslo downloads complete ---


In [7]:
#@title 📊 Step 3: Process and Export CSV
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import zipfile

file_pattern = f'{selected_city.lower()}_*.nc'
file_list = sorted(glob.glob(file_pattern))
extract_dir = 'extracted_temp'
os.makedirs(extract_dir, exist_ok=True)
all_data = []

print(f"--- Processing {len(file_list)} files for {selected_city} ---")

for f in file_list:
    try:
        # Check for ZIP signature 'PK'
        with open(f, 'rb') as test_f:
            is_zip = test_f.read(2) == b'PK'

        if is_zip:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            inner_files = glob.glob(os.path.join(extract_dir, "*.nc"))
            if inner_files:
                ds = xr.open_dataset(inner_files[0], engine='netcdf4')
                df = ds.to_dataframe().reset_index()
                all_data.append(df)
                for extra in inner_files: os.remove(extra)
        else:
            ds = xr.open_dataset(f, engine='netcdf4')
            df = ds.to_dataframe().reset_index()
            all_data.append(df)
        print(f"✅ Processed: {f}")
    except Exception as e:
        print(f"❌ Error on {f}: {e}")

if all_data:
    df_combined = pd.concat(all_data).sort_values('valid_time')
    df_combined['valid_time'] = pd.to_datetime(df_combined['valid_time'])

    # 1. Formatting and Unit Conversions
    df_combined['Date'] = df_combined['valid_time'].dt.strftime('%Y-%m-%d')
    df_combined['Hour'] = df_combined['valid_time'].dt.strftime('%H:00')
    df_combined['Temperature_C'] = df_combined['t2m'] - 273.15
    df_combined['Wind_Speed_ms'] = np.sqrt(df_combined['u10']**2 + df_combined['v10']**2)
    df_combined['GHI_Wm2'] = df_combined['ssrd'] / 3600

    # 2. Final Export
    final_output = df_combined[['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2']]
    csv_name = f'{selected_city}_Weather_Hourly_2024.csv'
    final_output.to_csv(csv_name, index=False)

    print(f"\nSUCCESS: Saved to {csv_name}")
    print(final_output.head())
else:
    print("No data frames found.")

--- Processing 13 files for Oslo ---
✅ Processed: oslo_2024_01.nc
✅ Processed: oslo_2024_02.nc
✅ Processed: oslo_2024_03.nc
✅ Processed: oslo_2024_04.nc
✅ Processed: oslo_2024_05.nc
✅ Processed: oslo_2024_06.nc
✅ Processed: oslo_2024_07.nc
✅ Processed: oslo_2024_08.nc
✅ Processed: oslo_2024_09.nc
✅ Processed: oslo_2024_10.nc
✅ Processed: oslo_2024_11.nc
✅ Processed: oslo_2024_12.nc
✅ Processed: oslo_2025_01.nc

SUCCESS: Saved to Oslo_Weather_Hourly_2024.csv
         Date   Hour  Temperature_C  Wind_Speed_ms    GHI_Wm2
0  2024-01-01  00:00      -4.939545       3.723875  107.74778
1  2024-01-01  01:00      -5.068451       3.770795    0.00000
2  2024-01-01  02:00      -5.149017       3.812386    0.00000
3  2024-01-01  03:00      -5.250824       3.806863    0.00000
4  2024-01-01  04:00      -5.350189       3.778243    0.00000
